# Power Analysis Demonstration

@author: Niral and Samanth

Adapted from lab 8 of the [Hardware Hacking Handbook Notebooks](https://github.com/HardwareHackingHandbook/notebooks)


In this notebook we will demonstrate how Simple Power Analysis can be used to crack RSA encryption. Revealing the secret key.

In [ ]:
# Some Basic imports
import numpy as np
import rsa
import random
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Label
from itertools import groupby
import matplotlib.pyplot as plt

# Set the seeds to ensure same results every time
np.random.seed(1338)
random.seed(1338)

# Leaky RSA Implementation

In [ ]:
#Normal Distribution for Multiplication and  mod operation (of the power). This is the mean and std dev
TIMING=(10,2.1)

#Implementing a leaky version of exponentiation
# EVery time a multiply and mod is taken the timer increments.
def leaky_exponentiation(secret, message, n):
    time = 0
    timeleak = [time]
    P = 1
    i = message

    while secret > 0:
        # print(secret)
        keybit = secret & 1
        if keybit:
            P = (P * i) % n
            time += np.random.normal(TIMING[0], TIMING[1])

        i = (i * i ) % n
        secret = secret >> 1
        time += np.random.normal(TIMING[0], TIMING[1])
        timeleak.append(time)
    return timeleak, P


def leaky_exponentiation_2(secret, message, n):
  time = 0
  timeleak = [time]
  P = 1
  s = message
  for i in range(secret.bit_length()):
    if (i > 0):
      s = (s * s) % n
      time += np.random.normal(TIMING[0], TIMING[1])
    if (secret & 1):
      P = (P * s) % n
      time += np.random.normal(TIMING[0], TIMING[1])

    timeleak.append(time)
    secret = secret >> 1



#RSA Encryption: enctyped_message = message^pub_exponent mod n
def rsa_encrypt(public_exponent, message, n):
    return pow(message, public_exponent, n)

#RSA Decryption that leaks some information about the private key.
# message = enctyped_message ^ private_exponent mod n
def rsa_decrypt_leaky(secret, encrypted_message, n):
    timeleak, message = leaky_exponentiation(secret, encrypted_message, n)
    return timeleak, message




 # Testing our RSA Encryption and Key generation


 In this section we show how to encrypt a message using RSA and decrypt it in code.

In [ ]:
# Testing out the functions

# Generating a 256 bit rsa key
key_size = 256
key = rsa.newkeys(key_size)


# Getting the modulus, public and private exponents
n = key[0].n
public_exponent = key[0].e
private_exponent = key[1].d


#This is Our Sample Message.
message = "Hello World!"
message_int = int.from_bytes(message.encode('utf-8'), 'big')
print("Message Represented in bytes: ",bin(message_int))
encrypted_message = rsa_encrypt(public_exponent, message_int, n)
print("Encrytped Message in bytes: ", bin(encrypted_message))

timeleak, decrypted_message = rsa_decrypt_leaky(private_exponent, encrypted_message, n)

byte_length = (decrypted_message.bit_length() + 7 )// 8
print("Decrypted Message ", decrypted_message.to_bytes(byte_length, 'big').decode('utf-8'))

# Simulating SPA traces of a leaky RSA

Here we create a simulated trace using the time leakages. We will use this trace to extract the key later!!!

In [ ]:
SAMPLES_PER_LOOP = 10
SCOPE_NOISE = 0.05

# Take the time leakages, and create a simulated trace
def timeleak_to_trace(timeleak, samples_per_loop, scope_noise):
    points = len(timeleak)*SAMPLES_PER_LOOP  # Trace length samples
    timeleak0 = np.asarray(timeleak) - timeleak[0] # Zero-base time
    timeidx = (timeleak0 / timeleak0[-1] * (points-1)).astype(int) # Convert to int indices
    trace = np.ones(points) # Set all to power value 1
    trace[timeidx] = 0 # Except when switch between operations

    # Add measurement noise
    trace += np.random.normal(0, scope_noise, len(trace))

    return trace, timeleak0

# Getting and Plotting the Trace

In [ ]:
trace, timeleak0 = timeleak_to_trace(timeleak, SAMPLES_PER_LOOP, SCOPE_NOISE)

# Plot the trace and try to interpret it yourself

In [ ]:
# Plot a trace
# Here is some nice code to plot it along with the true bits.
TRIM = 200  # Initial zoom in samples

trace, timeleak0 = timeleak_to_trace(timeleak, SAMPLES_PER_LOOP, SCOPE_NOISE)

# Plot simulated trace
output_notebook()
p = figure(x_range=(0,TRIM), x_axis_label='"Time"', y_axis_label='"Volt"', title="RSA Decryption trace with actual private key bits")

p.line(list(range(len(trace))), trace, line_color="blue")

# Plot known bits
op_idx = timeleak0 / timeleak0[-1] * (len(trace)-1)+0.5
for bit in range(len(op_idx)):
    p.add_layout(Label(x=op_idx[bit], y=0.8, text=str(private_exponent>>bit&1), text_color="red"))

show(p)

It is clear that relatively longer times between dips correpond to the bit one
while relatively shorter time between dips correspond to the bit zero.
Now let us try to use this information to extract the bits of the key from the trace

So we convert it to a list of duration between peaks

In [ ]:
def trace_to_difftime(trace):
  # Conver the trace to binary using a particular threshold, give the lengths of each peak as well
  # trace_rle: durations between each peaks
  # trace binary: 1 when above threshold, zero when below
  # threshold: The threshold below which we set zero and above we set one. (What should we set it to?)
  trace_rle = []
  trace_bin = []
  threshold = 0

  # YOUR CODE HERE

  return trace_rle, trace_bin, threshold

In [ ]:
# Let's plot the binary trace, threshold, and distances between peaks
trace_rle, trace_bin, threshold = trace_to_difftime(trace)

# Init figure
output_notebook()
p = figure(x_range=(0,TRIM), x_axis_label='"Time"', y_axis_label='"Binary"', title="RSA decryption binary trace with peak distances at given threshold")

# Binary trace
p.line(list(range(len(trace))), trace_bin, line_color="blue")

# Threshold line
p.line([0, len(trace)], [threshold,threshold], line_width=4, color='black', line_dash='dashed')

# Plot distances
for idx in range(len(trace_rle)):
    p.add_layout(Label(x=op_idx[idx], y=0.8, text=str(trace_rle[idx]), text_color="green"))

show(p)

In [ ]:
# This performs the Simple Power Analysis by turning given traces into peak distances,
# and in turn turning these distances into bits of guessed nonce k.
def simple_power_analysis(trace):
  # Analyse the trace return the following things:
  closest = [] # Array of the indices of the bits closest to the threshold (most doubtful)
  gussed_k_base = 0 # Integer of the guess
  difftime, _, _ = trace_to_difftime(trace)
  cutoff = 0 # The cutoff for time (above which bit is one and blow which bit is set to zero)
  # This is the heart of this attack

  # YOUR CODE HERE

  return closest, guessed_k_base, difftime, cutoff

In [ ]:
# Create a power-like trace
_, _, difftime, cutoff = simple_power_analysis(trace)

In [ ]:
# Let's have a look at the two distributions of time for double only or double+add
BINS=7 # Number of bins per histogram

bitactual = np.asarray([int(private_exponent & (1<<x) > 0) for x in range(0,len(difftime))]) # Turn k into bit vector

# Plot two histograms and cutoff
output_notebook()
p = figure(x_axis_label="Duration", y_axis_label="Count", title="Distributions for Double vs Double+Add, shown with cutoff")

# Calc histograms for bits 0 and 1
hist_0,edges_0 = np.histogram(difftime[bitactual==0], bins=BINS)
hist_1,edges_1 = np.histogram(difftime[bitactual==1], bins=BINS)

# Plot histograms
p.quad(top=hist_0,bottom=0,left=edges_0[:-1],right=edges_0[1:],fill_color='red',line_color='black', fill_alpha=0.5)
p.quad(top=hist_1,bottom=0,left=edges_1[:-1],right=edges_1[1:],fill_color='blue',line_color='black', fill_alpha=0.5)

# Add cutoff as vertical line
p.line([cutoff, cutoff], [0,np.max([hist_0, hist_1])], line_width=4, color='black', line_dash='dashed')

show(p)

In [ ]:

# Since we have some overlapping distributions, we'll have bit errors we'll need to bruteforce
# This function does a bruteforce of the bits closest to the decision boundary


def bruteforce(bits, guessed_private_exponent_base, closest, encrypted_message, n, private_exponent):
    # Take the bits closest bits, considering each possible configuration of flipping them.
    # Flip them and check if the key is correct (using the given private exponent, in practice you would do it by decrypting the message and looking at it), Return the correct key if found.

    # Guessed_Private_exponent_base: The guess made by the threshold
    # closest: Indices of the bits closest to threshold (most doubtful)
    # n, private_exponent, encrypted_message: from RSA
    # bits: No of bits to bruteforce

    # YOUR CODE HERE
    for i in range(2**bits):
      gussed_pe = guessed_private_exponent_base
      for j in range(bits):
        if (i & (1 << j)):
          gussed_pe = gussed_pe ^ (1 << closest[j])
      if gussed_pe == private_exponent:
        print("Found key ", bin(gussed_pe))



    ##

In [ ]:
MAX_ATTEMPTS = 200
BRUTEFORCE_BITS = 10


for attempt in range(MAX_ATTEMPTS):
    print("\n\n Attempt", attempt)

    # Perform a leaky rsa encryption and decryption
    # Testing out the functions
    message = "Hello World!"
    message_int = int.from_bytes(message.encode('utf-8'), 'big')
    encrypted_message = rsa_encrypt(public_exponent, message_int, n)
    timeleak, decrypted_message = rsa_decrypt_leaky(private_exponent, encrypted_message, n)
    byte_length = (decrypted_message.bit_length() + 7 )// 8
    print(decrypted_message.to_bytes(byte_length, 'big').decode('utf-8'))

    trace, _ = timeleak_to_trace(timeleak, SAMPLES_PER_LOOP, SCOPE_NOISE)
    closest, guessed_k_base, difftime, cutoff = simple_power_analysis(trace)
    print("Guessed k: ", bin(guessed_k_base))
    key = bruteforce(BRUTEFORCE_BITS, guessed_k_base, closest, encrypted_message, n, private_exponent)

    if key == private_exponent:
        print("Yes, Key Found: ", bin(key))
        break